In [249]:
from datetime import datetime
from datetime import date
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from faker import Faker
import json
from bson import Decimal128 as Decimal
from bson.objectid import ObjectId

In [4]:
c = MongoClient(host="localhost", port=27017)
print(f"connection established. {c}")
dbh = c["mydb"] #database handle

connection established. MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


In [13]:
[u for u in dbh.users.find()]

[{'_id': ObjectId('6113b890de24031ce926cbe2'),
  'username': 'janedoe',
  'firstname': 'Jane',
  'surname': 'Doe',
  'dateofbirth': datetime.datetime(1974, 4, 12, 0, 0),
  'email': 'janedoe74@example.com',
  'score': 0},
 {'_id': ObjectId('6113bbc55bfe5dcbbf930e7b'),
  'username': 'janedoe',
  'firstname': 'Jane',
  'surname': 'Doe',
  'dateofbirth': datetime.datetime(1974, 4, 12, 0, 0),
  'email': 'janedoe74@example.com',
  'score': 0}]

In [11]:
user_doc = {
        "username" : "janedoe",
        "firstname" : "Jane",
        "surname" : "Doe",
        "dateofbirth" : datetime(1974, 4, 12),
        "email" : "janedoe74@example.com",
        "score" : 0
}


In [12]:
dbh.users.insert_one(user_doc)

In [28]:
Faker().image()

['__class__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_factories',
 '_factory_map',
 '_locales',
 '_map_provider_method',
 '_select_factory',
 '_unique_proxy',
 '_weights',
 'aba',
 'add_provider',
 'address',
 'administrative_unit',
 'am_pm',
 'android_platform_token',
 'ascii_company_email',
 'ascii_email',
 'ascii_free_email',
 'ascii_safe_email',
 'bank_country',
 'bban',
 'binary',
 'boolean',
 'bothify',
 'bs',
 'building_number',
 'cache_pattern',
 'catch_phrase',
 'century',
 'chrome',
 'city',
 'city_prefix',
 'city_suffix',
 'color',
 'color_name',
 'company',
 'company_email',
 'company_suffix',
 '

In [292]:
dbh.users.find_one()

{'_id': ObjectId('6113fcb75bfe5dcbbf930e7e'),
 'job': 'Scientist, research (life sciences)',
 'company': 'Arias LLC',
 'ssn': '168-88-7705',
 'residence': '355 Marisa Isle\nKevinberg, WV 82588',
 'current_location': [Decimal128('20.8178755'), Decimal128('87.801446')],
 'blood_group': 'O-',
 'website': ['http://lang.com/'],
 'username': 'marcus33',
 'name': 'Anthony Hall',
 'sex': 'M',
 'address': '205 James Haven Apt. 337\nNew Alexander, SD 52214',
 'mail': 'belljessica@hotmail.com',
 'birthdate': datetime.datetime(1919, 6, 14, 0, 0),
 'age': 102}

In [235]:
def gen_profile():
    x = Faker().profile()
    x['current_location'] = (Decimal(x['current_location'][0]), Decimal(x['current_location'][1]))
    x['age'] = date.today().year - x['birthdate'].year
    x['birthdate'] = datetime(x['birthdate'].year, x['birthdate'].month, x['birthdate'].day)
    return x
gen_profile()

{'job': 'Press sub',
 'company': 'Thompson, Hicks and Baker',
 'ssn': '473-81-0427',
 'residence': '2930 Smith Parkways Suite 741\nBrandontown, MS 85238',
 'current_location': (Decimal128('-3.125877'), Decimal128('95.360536')),
 'blood_group': 'O-',
 'website': ['http://www.chen.com/',
  'https://thompson-turner.biz/',
  'https://price.org/',
  'http://lewis.com/'],
 'username': 'yjackson',
 'name': 'Denise Fisher',
 'sex': 'F',
 'address': '6093 Kathleen Brook Apt. 221\nNorth Michael, NY 53884',
 'mail': 'nicholas59@gmail.com',
 'birthdate': datetime.datetime(1945, 7, 28, 0, 0),
 'age': 76}

In [213]:
import tqdm

In [236]:
for i in tqdm.tqdm(range(1000)):
    dbh.users.insert_one(gen_profile())

100%|██████████| 1000/1000 [00:48<00:00, 20.70it/s]


In [281]:
[u for u in dbh.users.find({
    "$and": [
                {"age":{"$gte":13}},
                {"age":{"$lte":19}}
            ]},
    {"_id":0, "age":1, "name":1, "teenager":1}).limit(10).sort('age')]

[{'name': 'Sandra Graves', 'age': 13, 'teenager': True},
 {'name': 'Renee Farmer', 'age': 13, 'teenager': True},
 {'name': 'Michael Whitehead', 'age': 13, 'teenager': True},
 {'name': 'Jesus Craig', 'age': 13, 'teenager': True},
 {'name': 'Jeff Jacobs', 'age': 13, 'teenager': True},
 {'name': 'Margaret Torres', 'age': 13, 'teenager': True},
 {'name': 'Kelsey Hendrix', 'age': 13, 'teenager': True},
 {'name': 'Katherine Ramirez', 'age': 14, 'teenager': True},
 {'name': 'Samuel Greene', 'age': 14, 'teenager': True},
 {'name': 'Sabrina Phillips', 'age': 14, 'teenager': True}]

In [255]:
ret = dbh.users.delete_one({'_id':ObjectId('6113b890de24031ce926cbe2')})

In [273]:
dbh.users.update_many({
    "$and": [
                {"age":{"$gte":13}},
                {"age":{"$lte":19}}
            ]},
    {
        "$set":{
                        "teenager":True
                },
        "$currentDate":{"lastModified":True}
                  
    })

In [289]:
dbh.users.count_documents({
    "$and": [
                {"age":{"$gte":100}},
                {"age":{"$lte":120}}
            ]})

137

In [294]:
dbh.users.distinct("blood_group")

['A+', 'A-', 'AB+', 'AB-', 'B+', 'B-', 'O+', 'O-']

In [316]:
documents = [{"_id": 1, "dept": "A",
                  "item": {"code": "012", "color": "red"},
                  "sizes": ["S", "L"]},
             {"_id": 2, "dept": "A",
                  "item": {"code": "012", "color": "blue"},
                  "sizes": ["M", "S"]},
             {"_id": 3, "dept": "B",
                  "item": {"code": "101", "color": "blue"},
                  "sizes": "L"},
             {"_id": 4, "dept": "A",
                  "item": {"code": "679", "color": "black"},
                  "sizes": ["M"]}]
dbh.distinct_t.insert_many(documents)

In [317]:
for doc in dbh.distinct_t.find({}):
    print(doc)

{'_id': 1, 'dept': 'A', 'item': {'code': '012', 'color': 'red'}, 'sizes': ['S', 'L']}
{'_id': 2, 'dept': 'A', 'item': {'code': '012', 'color': 'blue'}, 'sizes': ['M', 'S']}
{'_id': 3, 'dept': 'B', 'item': {'code': '101', 'color': 'blue'}, 'sizes': 'L'}
{'_id': 4, 'dept': 'A', 'item': {'code': '679', 'color': 'black'}, 'sizes': ['M']}


In [306]:
dbh.distinct_t.distinct("item.code", {"dept":"B"})

['101']

In [309]:
dbh.distinct_t.find_one_and_replace({"dept":"A"}, {"dept":"C"})

{'_id': 1,
 'dept': 'A',
 'item': {'code': '012', 'color': 'red'},
 'sizes': ['S', 'L']}

In [314]:
dbh.distinct_t.delete_many({})

In [324]:
aggr = dbh.users.aggregate(
[
    {
        "$group": {
            "_id": "$blood_group",
            "total": {"$sum":1}
        }
    }
]
)

for i in aggr:
    print(i)

{'_id': 'A-', 'total': 131}
{'_id': 'B-', 'total': 104}
{'_id': 'AB-', 'total': 116}
{'_id': 'A+', 'total': 132}
{'_id': 'AB+', 'total': 126}
{'_id': 'B+', 'total': 130}
{'_id': 'O-', 'total': 134}
{'_id': 'O+', 'total': 127}
